## Deep Research

One of the classic cross-business Agentic use cases!
A Deep Research agent is broadly applicable to any business area, and to your own day-to-day activities.

In [10]:
from agents import Agent, WebSearchTool, trace, Runner, gen_trace_id, function_tool
from agents.model_settings import ModelSettings
from pydantic import BaseModel, Field
from dotenv import load_dotenv
import asyncio
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
from typing import Dict
from IPython.display import display, Markdown

In [11]:
load_dotenv(override=True)

True

In [12]:
# Configuration & credentials
llm_model = "gpt-4o-mini"
sender_email = os.environ["SENDER_EMAIL"]
receiver_email = os.environ["RECEIVER_EMAIL"]

## OpenAI Hosted Tools

OpenAI Agents SDK includes the following hosted tools:

The `WebSearchTool` lets an agent search the web.  
The `FileSearchTool` allows retrieving information from your OpenAI Vector Stores.  
The `ComputerTool` allows automating computer use tasks like taking screenshots and clicking.

In [13]:
research_instructions = """You are a research assistant. Given a search term,
you search the web for that term and produce a concise summary of the results.
The summary must be less than 100 words. Capture the main points.
This will be consumed by someone synthesizing a report, so it's vital you capture
the essence and ignore any fluff. Do not include any additional commentary other
than the summary itself."""

search_agent = Agent(name="Search agent",
                     instructions=research_instructions,
                     tools=[WebSearchTool(search_context_size="low")],
                     model=llm_model,
                     model_settings=ModelSettings(tool_choice="required"))

### We will now use Structured Outputs, and include a description of the fields

In [14]:
HOW_MANY_SEARCHES = 3

planner_instructions = (f"You are a helpful research assistant. Given a query, "
                        f"come up with a set of web searches to respond with the best answer of the user query. Output {HOW_MANY_SEARCHES} terms to query for.")

# Use Pydantic to define the Schema of our response - this is known as "Structured Outputs"
class WebSearchItem(BaseModel):
    reason: str = Field(description="Your reasoning for why this search is important to the query.")
    query: str = Field(description="The search term to use for the web search.")


class WebSearchPlan(BaseModel):
    searches: list[WebSearchItem] = Field(description="A list of web searches to perform to best answer the query.")


planner_agent = Agent(name="PlannerAgent",
                      instructions=planner_instructions,
                      model=llm_model,
                      output_type=WebSearchPlan)

In [15]:
@function_tool
def send_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email(sender_email) # Change this to your verified email
    to_email = To(receiver_email) # Change this to your email
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    response = sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

In [16]:
email_instructions = """You are able to send a nicely formatted HTML email based on a detailed report. You will be provided with a detailed report. You should use your tool to send one email, providing the report converted into clean, well presented HTML with an appropriate subject line."""

email_agent = Agent(name="Email agent",
                    instructions=email_instructions,
                    tools=[send_email],
                    model=llm_model)

In [17]:
writer_instructions = """You are a senior researcher tasked with writing a cohesive report for a research query. You will be provided with the original query, and some initial research done by a research assistant.

You should first come up with an outline for the report that describes the structure and
flow of the report. Then, generate the report and return that as your final output. The
final output should be in markdown format, and it should be lengthy and detailed. Aim for
1-2 pages of content, at least 200 words."""


class ReportData(BaseModel):
    short_summary: str = Field(description="A short 2-3 sentence summary of the findings.")

    markdown_report: str = Field(description="The final report")

    follow_up_questions: list[str] = Field(description="Suggested topics to research further")


writer_agent = Agent(
    name="WriterAgent",
    instructions=writer_instructions,
    model=llm_model,
    output_type=ReportData,
)

### The next 3 functions will plan and execute the search, using planner_agent and search_agent

In [18]:
async def plan_searches(user_query: str):
    """ Use the planner_agent to plan which searches to run for the query """
    print("Planning searches...")
    result = await Runner.run(planner_agent, f"Query: {user_query}")
    print(f"Will perform {len(result.final_output.searches)} searches")
    return result.final_output

async def perform_searches(plan_search: WebSearchPlan):
    """ Call search() for each item in the search plan """
    print("Searching...")
    tasks = [asyncio.create_task(search(item)) for item in plan_search.searches]
    results = await asyncio.gather(*tasks)
    print("Finished searching")
    return results

async def search(item: WebSearchItem):
    """ Use the search agent to run a web search for each item in the search plan """
    search_input = f"Search term: {item.query}\nReason for searching: {item.reason}"
    result = await Runner.run(search_agent, search_input)
    return result.final_output

### The next 2 functions write a report and email it

In [19]:
async def write_report(user_query: str, search_result: list[str]):
    """ Use the writer agent to write a report based on the search results"""
    print("Thinking about report...")
    writer_input = f"Original query: {user_query}\nSummarized search results: {search_result}"
    result = await Runner.run(writer_agent, writer_input)
    print("Finished writing report")
    return result.final_output

async def send_email(writer_report: ReportData):
    """ Use the email agent to send an email with the report """
    print("Writing email...")
    result = await Runner.run(email_agent, writer_report.markdown_report)
    print("Email sent")
    return writer_report

### Showtime!

In [20]:
query ="Latest AI Agent frameworks in 2025"

with trace("Research trace"):
    print("Starting research...")
    search_plan = await plan_searches(query)
    search_results = await perform_searches(search_plan)
    report = await write_report(query, search_results)
    await send_email(report)

Starting research...
Planning searches...
Will perform 3 searches
Searching...
Finished searching
Thinking about report...
Finished writing report
Writing email...
Email sent
Hooray!


### As always, take a look at the trace

https://platform.openai.com/traces

 "<h1>Latest AI Agent Frameworks in 2025</h1>\n<h2>Introduction</h2>\n<p>The landscape of AI agent frameworks has evolved rapidly in 2025, showcasing a variety of solutions tailored to specific application needs. This report summarizes the latest frameworks that have gained traction this year, driven by advancements from leading tech companies. The ability to build and deploy intelligent agents has become crucial across industries, ranging from enterprise solutions to decentralized applications.</p>\n<h2>Key AI Agent Frameworks</h2>\n<p>In 2025, several prominent frameworks have emerged, each offering unique features and capabilities:</p>\n<h3>1. Amazon Bedrock AgentCore</h3>\n<strong>Provider</strong>: Amazon Web Services (AWS)<br/>\n<p>Amazon Bedrock AgentCore is designed to simplify the development and deployment of advanced AI agents. It includes modular services that enhance scalability, including the AgentCore Runtime for serverless deployment and AgentCore Memory for effective context management. The platform aims to support enterprise-scale applications, making it a suitable choice for businesses looking to integrate AI agents. <a href=\"https://www.techradar.com/pro/aws-looks-to-super-charge-ai-agents-with-amazon-bedrock-agentcore?utm_source=openai\">(TechRadar)</a></p>\n<h3>2. AutoGen</h3>\n<strong>Provider</strong>: Microsoft<br/>\n<p>AutoGen specializes in multi-agent orchestration, allowing for autonomous, event-driven systems to manage complex tasks seamlessly. This framework emphasizes the coordination of multiple agents, enabling them to function together efficiently in dynamic environments. <a href=\"https://www.linkedin.com/pulse/ai-agent-frameworks-june-2025-comprehensive-overview-chadi-abi-fadel-wcu5c?utm_source=openai\">(LinkedIn)</a></p>\n<h3>3. LangChain</h3>\n<strong>Features</strong>: Multimodal Application Framework<br/>\n<p>LangChain serves as a robust framework for building applications powered by large language models (LLMs). It addresses challenges such as context retention and executing multi-step tasks, making it a versatile choice for application developers. <a href=\"https://medium.com/%40elisowski/top-ai-agent-frameworks-in-2025-9bcedab2e239?utm_source=openai\">(Medium)</a></p>\n<h3>4. LangGraph</h3>\n<strong>Extension</strong>: For Stateful Systems<br/>\n<p>An extension of LangChain, LangGraph specializes in managing stateful, multi-actor systems. It is particularly suited for complex workflows requiring adaptive AI applications, offering graph-based workflows that enhance collaboration among agents. <a href=\"https://medium.com/%40elisowski/top-ai-agent-frameworks-in-2025-9bcedab2e239?utm_source=openai\">(Medium)</a></p>\n<h3>5. CrewAI</h3>\n<strong>Focus</strong>: Role-based Agents<br/>\n<p>CrewAI is tailored for environments that prioritize collaboration and specialized expertise. Its role-based approach allows teams to work more effectively on shared projects, making it useful in collaborative problem-solving contexts. <a href=\"https://medium.com/%40elisowski/top-ai-agent-frameworks-in-2025-9bcedab2e239?utm_source=openai\">(Medium)</a></p>\n<h3>6. Eliza (Web3 Integration)</h3>\n<strong>Specialization</strong>: Decentralized Applications<br/>\n<p>Eliza is a Web3-friendly operating system for AI agents, streamlining the integration of AI with decentralized applications. This development supports the growing trend of blockchain technology in AI. <a href=\"https://arxiv.org/abs/2501.06781?utm_source=openai\">(ArXiv)</a></p>\n<h3>7. AgentMesh</h3>\n<strong>Utility</strong>: Software Development Automation<br/>\n<p>AgentMesh leverages multiple cooperating LLM-powered agents to automate software development tasks, enhancing efficiency in the software engineering process. This cooperative structure enables a more streamlined development workflow. <a href=\"https://arxiv.org/abs/2507.19902?utm_source=openai\">(ArXiv)</a></p>\n<h3>8. Agent Lightning</h3>\n<strong>Capability</strong>: Reinforcement Learning<br/>\n<p>Agent Lightning offers an extensible framework designed for reinforcement learning-based training, achieving separation between agent execution and training processes, thus fostering innovative solution development. <a href=\"https://arxiv.org/abs/2508.03680?utm_source=openai\">(ArXiv)</a></p>\n<h3>9. Model Context Protocol (MCP)</h3>\n<p>The adoption of the Model Context Protocol by organizations such as OpenAI, Google DeepMind, and Microsoft signifies a move towards standardized AI tool connectivity and interoperability, enhancing the collaborative potential of AI tools across platforms. <a href=\"https://en.wikipedia.org/wiki/Model_Context_Protocol?utm_source=openai\">(Wikipedia)</a></p>\n<h2>Conclusion</h2>\n<p>The advancements in AI agent frameworks in 2025 showcase a commitment by major tech players to facilitate more versatile AI solutions. Whether for enterprise applications, collaborative projects, or decentralized systems, these frameworks enhance the capabilities of AI, promising to drive innovation forward. As the technology matures, we can expect further integrations and improvements that will shape how AI agents interact and function across various industries.</p>\n<h2>Follow-Up Questions</h2>\n<ul>\n<li>What are the potential applications of these AI agent frameworks in specific industries?</li>\n<li>How do the features of these frameworks compare for small vs. large enterprises?</li>\n<li>What are the security implications of deploying AI agents in decentralized systems?</li>\n<li>How can organizations leverage multiple frameworks for enhanced productivity?</li>\n<li>What future developments can we anticipate in AI agent protocols beyond 2025?</li>\n</ul>"
})